# Disciplina: Introdução à Inteligência Artificial
Instituto Federal do Norte de Minas Gerais Campus Montes Claros <br/>
Curso: Ciência da Computação <br/>
Aluno: David Walter Jansen <br/>

**Experimento 1: um algoritmo genético simples, sem cruzamento**

1. Geração da população inicial, de tamanho N, de forma aleatória, entre -5.12 e 5.12. Sugestão N=20
2. Seleção por torneio 2 a 2. Lembrando que os indivíduos selecionados podem ser repetidos.
3. Realizar a mutação, sendo que, a taxa de mutação seja de 30% (por exemplo, 30 a cada 100 indivíduos devem sofrer mutação). 
- Da população selecionada no item 2, você deve escolher 30% deles e realizar uma perturbação de + ou - 5% do valor do indivíduo. Por exemplo, selecione um indivíduo, ele deve contar x_1 e x_2:
1. Selecione qual variável/dimensão você deseja alterar
2. Selecione qual operação você irá realizar +5% ou -5%

4. Entre os pais e filhos, escolha os novos N indivíduos, que irão para a próxima geração
5. Exiba a cada geração o melhor indivíduo (x e f(x))
6. Repita 2 a 5 até atingir o número máximo de gerações. Sugestão: 100 gerações.

Depois do algoritmo pronto, execute-o 30 vezes. Para cada execução salve o melhor indivíduo ao final das gerações. Portanto, serão 30 indivíduos diferentes

In [24]:
import numpy as np
import random
import math

In [25]:
num_ger = 100 #numero máximo de gerações que AG executará
num_pop = 20  #haverá 20 individuos na populacao
chancem = 0.3 #30% dos individuos selecionados sofrerão mutação
num_dim=2
xmin=-5.12
xmax=5.12

In [26]:
#define a função para gerar a pop inicial de forma aleatória
def gera_pop_inicial(num_pop, n_dim,xmin, xmax):
    return np.random.uniform(xmin,xmax,(num_pop,num_dim))

In [27]:
def inverte(fx):
    hx = min(fx);
    desl = 10**-0.2 - hx;
    return 1/( fx + desl);

def rastrigin(X, **kwargs):
    
    A = kwargs.get('A')
    [row, col] = X.shape
    fx= np.zeros(row)
    for i in range(row):
        for j in range(col):
            fx[i] += X[i,j]**2 - A * np.cos(2 * math.pi * X[i,j])
    #print(X)
    return A*col + fx

In [28]:
def popTorneio(ind, X):
    [row, col] = X.shape
    Xt = np.zeros((row,col))
    
    for i in range(row):
        Xt[i] = X[ind[i].astype(int)]
        
    return Xt

In [29]:
def torneio(fx):
    ind = np.zeros(len(fx))
    for i in range(len(fx)):
        #sorteia dois individuos e seleciona o melhor com relação ao fitness, armazena o indice do vendedor
        i1 = random.randint(0,len(fx)-1)
        i2 = random.randint(0,len(fx)-1)
        #print(i1,i2)
        if fx[i1] > fx[i2]:
            ind[i] = i1
        else:
            ind[i] = i2
    return ind

In [30]:
def mutacao(X, chance):
    [row, col] = X.shape
    Xn = np.zeros((row,col))
    for i in range(row):
        #sorteia um individuo e verifica se ele deve ser mutado
        i1 = random.randint(0,row-1)
        
        if random.random() <= chance:
            #print(i1)
            if random.random()>0.5:
                Xn[i,:] = X[i,:]*1.05
            else:
                Xn[i,:] = X[i,:]*-1.05 
        else:
            Xn[i,:] = X[i,:]
    return Xn

In [31]:
def update(X, z, Xn, fxn, taxa):
    auxIndX = np.argsort(z) #indices da ordenação de z
    auxIndXn = np.argsort(fxn) #indices da ordenação de fxn
    
    alterar = round(taxa*num_pop) #quantidade de individuos alterados para a proxima geração.
    
    #substitui os piores da geração atual, pelos melhores da geração anterior
    for i in range(alterar):
        Xn[auxIndXn[i].astype(int)] = X[auxIndX[i].astype(int)]
        
    return Xn

In [32]:
#armazena o melhor elemento de cada geração
melhoresIndividuos = np.zeros((30,2))
melhoresFx = np.zeros((30,2))
    
for exect in range(30):
    print("\n----------%dº Execução----------\n" %(exect + 1))
    #Passo 1: geração da população inicial
    X = gera_pop_inicial(num_pop,num_dim, xmin, xmax)

    for i in range(num_ger):
        #agora devemos avaliar todos os individuos da população inicial
        z = rastrigin(X, A=10)
        fx = inverte(z)
        
        #descomentar as duas linhas abaixo para mostrar o melhor a cada geração
        melhorind =  np.argmin(z)
        melhorfx = fx[melhorind]
        
        print("%dº geração: " %(i+1))
        print("Melhor individuo:", end=' ')
        print(X[melhorind])
        print("Fx para esse individuo:", end=' ')
        print(melhorfx, end='\n')

        #Passo 2: realização do torneio
        ind = torneio(fx)
        Xt = popTorneio(ind, X)

        #Passo 3, 4, 5 e 6: nova população após a mutação
        Xn = mutacao(Xt, chancem) 
        zn = rastrigin(Xn, A=10)
        fxn = inverte(zn)

        #Atualização
        taxa = 0.05 #5%
        X = update(X, z, Xn, fxn, taxa)
    
    #armazena o melhor da g-ésima execução após 100 gerações. Nesse caso, será da função, ao inves da inversão, 
    #pois a inversão depende do mínimo este pode variar a cada iteração 
    z = rastrigin(X, A=10)
    fx = inverte(z)
    melhorind =  np.argmin(z)
    melhoresIndividuos[exect] = X[melhorind]
    melhorfx = fx[melhorind]
    melhoresFx[exect] = melhorfx


----------1º Execução----------

1º geração: 
Melhor individuo: [-1.94506386  0.18241091]
Fx para esse individuo: 1.5848931924611112
2º geração: 
Melhor individuo: [-1.94506386  0.18241091]
Fx para esse individuo: 1.5848931924611112
3º geração: 
Melhor individuo: [-1.94506386  0.18241091]
Fx para esse individuo: 1.5848931924611112
4º geração: 
Melhor individuo: [-1.94506386  0.18241091]
Fx para esse individuo: 1.5848931924611112
5º geração: 
Melhor individuo: [-1.94506386  0.18241091]
Fx para esse individuo: 1.5848931924611112
6º geração: 
Melhor individuo: [-1.94506386  0.18241091]
Fx para esse individuo: 1.5848931924611112
7º geração: 
Melhor individuo: [-1.94506386  0.18241091]
Fx para esse individuo: 1.5848931924611112
8º geração: 
Melhor individuo: [-1.94506386  0.18241091]
Fx para esse individuo: 1.5848931924611112
9º geração: 
Melhor individuo: [-1.94506386  0.18241091]
Fx para esse individuo: 1.5848931924611112
10º geração: 
Melhor individuo: [-1.94506386  0.18241091]
Fx para 

64º geração: 
Melhor individuo: [-0.03602541  1.94881585]
Fx para esse individuo: 1.5848931924611134
65º geração: 
Melhor individuo: [-0.03602541  1.94881585]
Fx para esse individuo: 1.5848931924611134
66º geração: 
Melhor individuo: [-0.03602541  1.94881585]
Fx para esse individuo: 1.5848931924611134
67º geração: 
Melhor individuo: [-0.03602541  1.94881585]
Fx para esse individuo: 1.5848931924611134
68º geração: 
Melhor individuo: [-0.03602541  1.94881585]
Fx para esse individuo: 1.5848931924611134
69º geração: 
Melhor individuo: [-0.03602541  1.94881585]
Fx para esse individuo: 1.5848931924611134
70º geração: 
Melhor individuo: [-0.03602541  1.94881585]
Fx para esse individuo: 1.5848931924611134
71º geração: 
Melhor individuo: [-0.03602541  1.94881585]
Fx para esse individuo: 1.5848931924611134
72º geração: 
Melhor individuo: [-0.03602541  1.94881585]
Fx para esse individuo: 1.5848931924611134
73º geração: 
Melhor individuo: [-0.03602541  1.94881585]
Fx para esse individuo: 1.5848931

Melhor individuo: [-2.03209797  1.9889164 ]
Fx para esse individuo: 1.5848931924611134
10º geração: 
Melhor individuo: [-2.03209797  1.9889164 ]
Fx para esse individuo: 1.5848931924611134
11º geração: 
Melhor individuo: [-2.03209797  1.9889164 ]
Fx para esse individuo: 1.5848931924611134
12º geração: 
Melhor individuo: [-2.03209797  1.9889164 ]
Fx para esse individuo: 1.5848931924611134
13º geração: 
Melhor individuo: [-2.03209797  1.9889164 ]
Fx para esse individuo: 1.5848931924611134
14º geração: 
Melhor individuo: [-2.03209797  1.9889164 ]
Fx para esse individuo: 1.5848931924611134
15º geração: 
Melhor individuo: [-2.03209797  1.9889164 ]
Fx para esse individuo: 1.5848931924611134
16º geração: 
Melhor individuo: [-2.03209797  1.9889164 ]
Fx para esse individuo: 1.5848931924611134
17º geração: 
Melhor individuo: [-2.03209797  1.9889164 ]
Fx para esse individuo: 1.5848931924611134
18º geração: 
Melhor individuo: [ 2.03209797 -1.9889164 ]
Fx para esse individuo: 1.5848931924611134
19º 

9º geração: 
Melhor individuo: [2.0050333  0.81608199]
Fx para esse individuo: 1.5848931924611112
10º geração: 
Melhor individuo: [2.0050333  0.81608199]
Fx para esse individuo: 1.5848931924611112
11º geração: 
Melhor individuo: [2.0050333  0.81608199]
Fx para esse individuo: 1.5848931924611112
12º geração: 
Melhor individuo: [2.0050333  0.81608199]
Fx para esse individuo: 1.5848931924611112
13º geração: 
Melhor individuo: [2.0050333  0.81608199]
Fx para esse individuo: 1.5848931924611112
14º geração: 
Melhor individuo: [2.0050333  0.81608199]
Fx para esse individuo: 1.5848931924611112
15º geração: 
Melhor individuo: [2.0050333  0.81608199]
Fx para esse individuo: 1.5848931924611112
16º geração: 
Melhor individuo: [2.0050333  0.81608199]
Fx para esse individuo: 1.5848931924611112
17º geração: 
Melhor individuo: [2.0050333  0.81608199]
Fx para esse individuo: 1.5848931924611112
18º geração: 
Melhor individuo: [2.0050333  0.81608199]
Fx para esse individuo: 1.5848931924611112
19º geração

Fx para esse individuo: 1.5848931924611112
9º geração: 
Melhor individuo: [0.19523407 1.96878218]
Fx para esse individuo: 1.5848931924611112
10º geração: 
Melhor individuo: [0.19523407 1.96878218]
Fx para esse individuo: 1.5848931924611112
11º geração: 
Melhor individuo: [0.19523407 1.96878218]
Fx para esse individuo: 1.5848931924611112
12º geração: 
Melhor individuo: [0.19523407 1.96878218]
Fx para esse individuo: 1.5848931924611112
13º geração: 
Melhor individuo: [0.19523407 1.96878218]
Fx para esse individuo: 1.5848931924611112
14º geração: 
Melhor individuo: [0.19523407 1.96878218]
Fx para esse individuo: 1.5848931924611112
15º geração: 
Melhor individuo: [0.19523407 1.96878218]
Fx para esse individuo: 1.5848931924611112
16º geração: 
Melhor individuo: [0.19523407 1.96878218]
Fx para esse individuo: 1.5848931924611112
17º geração: 
Melhor individuo: [0.19523407 1.96878218]
Fx para esse individuo: 1.5848931924611112
18º geração: 
Melhor individuo: [0.19523407 1.96878218]
Fx para ess

...

Fx para esse individuo: 1.5848931924611112
3º geração: 
Melhor individuo: [-0.05971118  3.01663287]
Fx para esse individuo: 1.5848931924611112
4º geração: 
Melhor individuo: [-0.05971118  3.01663287]
Fx para esse individuo: 1.5848931924611112
5º geração: 
Melhor individuo: [-0.05971118  3.01663287]
Fx para esse individuo: 1.5848931924611112
6º geração: 
Melhor individuo: [-0.05971118  3.01663287]
Fx para esse individuo: 1.5848931924611112
7º geração: 
Melhor individuo: [-0.05971118  3.01663287]
Fx para esse individuo: 1.5848931924611112
8º geração: 
Melhor individuo: [-0.05971118  3.01663287]
Fx para esse individuo: 1.5848931924611112
9º geração: 
Melhor individuo: [-0.05971118  3.01663287]
Fx para esse individuo: 1.5848931924611112
10º geração: 
Melhor individuo: [-0.05971118  3.01663287]
Fx para esse individuo: 1.5848931924611112
11º geração: 
Melhor individuo: [-0.05971118  3.01663287]
Fx para esse individuo: 1.5848931924611112
12º geração: 
Melhor individuo: [-0.05971118  3.0166328

Melhor individuo: [-3.95571407  3.00485022]
Fx para esse individuo: 1.5848931924611112
3º geração: 
Melhor individuo: [-3.95571407  3.00485022]
Fx para esse individuo: 1.5848931924611112
4º geração: 
Melhor individuo: [ 0.90702753 -2.82153989]
Fx para esse individuo: 1.5848931924611112
5º geração: 
Melhor individuo: [-0.90702753  2.82153989]
Fx para esse individuo: 1.5848931924611112
6º geração: 
Melhor individuo: [ 0.95237891 -2.96261688]
Fx para esse individuo: 1.5848931924611158
7º geração: 
Melhor individuo: [ 0.95237891 -2.96261688]
Fx para esse individuo: 1.5848931924611158
8º geração: 
Melhor individuo: [-0.95237891  2.96261688]
Fx para esse individuo: 1.5848931924611158
9º geração: 
Melhor individuo: [ 0.95237891 -2.96261688]
Fx para esse individuo: 1.5848931924611158
10º geração: 
Melhor individuo: [ 0.95237891 -2.96261688]
Fx para esse individuo: 1.5848931924611158
11º geração: 
Melhor individuo: [ 0.95237891 -2.96261688]
Fx para esse individuo: 1.5848931924611158
12º geração

Melhor individuo: [ 0.91998502 -0.84834029]
Fx para esse individuo: 1.5848931924611134
3º geração: 
Melhor individuo: [ 0.91998502 -0.84834029]
Fx para esse individuo: 1.5848931924611134
4º geração: 
Melhor individuo: [ 0.91998502 -0.84834029]
Fx para esse individuo: 1.5848931924611134
5º geração: 
Melhor individuo: [ 0.96598428 -0.89075731]
Fx para esse individuo: 1.5848931924611134
6º geração: 
Melhor individuo: [ 1.01428349 -0.93529517]
Fx para esse individuo: 1.5848931924611134
7º geração: 
Melhor individuo: [-1.01428349  0.93529517]
Fx para esse individuo: 1.5848931924611134
8º geração: 
Melhor individuo: [ 1.01428349 -0.93529517]
Fx para esse individuo: 1.5848931924611134
9º geração: 
Melhor individuo: [ 1.01428349 -0.93529517]
Fx para esse individuo: 1.5848931924611134
10º geração: 
Melhor individuo: [ 1.01428349 -0.93529517]
Fx para esse individuo: 1.5848931924611134
11º geração: 
Melhor individuo: [ 1.01428349 -0.93529517]
Fx para esse individuo: 1.5848931924611134
12º geração

Fx para esse individuo: 1.5848931924611112
2º geração: 
Melhor individuo: [-0.32923815 -2.15863733]
Fx para esse individuo: 1.5848931924611112
3º geração: 
Melhor individuo: [-1.15784138  3.99339124]
Fx para esse individuo: 1.5848931924611112
4º geração: 
Melhor individuo: [-1.15784138  3.99339124]
Fx para esse individuo: 1.5848931924611112
5º geração: 
Melhor individuo: [-1.15784138  3.99339124]
Fx para esse individuo: 1.5848931924611112
6º geração: 
Melhor individuo: [-1.15784138  3.99339124]
Fx para esse individuo: 1.5848931924611112
7º geração: 
Melhor individuo: [ 1.15784138 -3.99339124]
Fx para esse individuo: 1.5848931924611112
8º geração: 
Melhor individuo: [ 1.15784138 -3.99339124]
Fx para esse individuo: 1.5848931924611112
9º geração: 
Melhor individuo: [ 1.15784138 -3.99339124]
Fx para esse individuo: 1.5848931924611112
10º geração: 
Melhor individuo: [ 1.15784138 -3.99339124]
Fx para esse individuo: 1.5848931924611112
11º geração: 
Melhor individuo: [ 1.15784138 -3.99339124

Fx para esse individuo: 1.5848931924611134
2º geração: 
Melhor individuo: [-0.85415129  0.09309049]
Fx para esse individuo: 1.5848931924611134
3º geração: 
Melhor individuo: [-0.89685885  0.09774501]
Fx para esse individuo: 1.5848931924611134
4º geração: 
Melhor individuo: [-0.94170179  0.10263226]
Fx para esse individuo: 1.5848931924611134
5º geração: 
Melhor individuo: [-0.94170179  0.10263226]
Fx para esse individuo: 1.5848931924611134
6º geração: 
Melhor individuo: [-0.98878688  0.10776388]
Fx para esse individuo: 1.5848931924611134
7º geração: 
Melhor individuo: [-0.98878688  0.10776388]
Fx para esse individuo: 1.5848931924611134
8º geração: 
Melhor individuo: [-0.98878688  0.10776388]
Fx para esse individuo: 1.5848931924611134
9º geração: 
Melhor individuo: [-0.98878688  0.10776388]
Fx para esse individuo: 1.5848931924611134
10º geração: 
Melhor individuo: [-0.98878688  0.10776388]
Fx para esse individuo: 1.5848931924611134
11º geração: 
Melhor individuo: [-0.98878688  0.10776388

# Resultado para cada um das 30 execuções:

In [34]:
melhoresIndividuos

array([[-1.94506386,  0.18241091],
       [-0.03602541,  1.94881585],
       [ 0.17784429,  0.04271407],
       [-2.03209797,  1.9889164 ],
       [ 2.0050333 ,  0.81608199],
       [ 0.19523407,  1.96878218],
       [ 0.93568344,  2.03810537],
       [-0.95679227,  2.04963861],
       [ 2.03765576,  2.00447809],
       [ 2.01215261,  0.07393854],
       [ 0.36422076,  0.9322758 ],
       [ 1.0328349 , -1.9145225 ],
       [-0.99602875, -1.97976364],
       [ 0.92310322, -3.0734955 ],
       [ 0.04663569,  4.02967769],
       [ 2.96474975,  0.17478961],
       [-1.97536005,  3.08746115],
       [-0.2223591 ,  0.21371634],
       [-3.0082719 , -0.06033321],
       [ 1.88387095,  2.02183402],
       [-0.0915187 ,  2.02987231],
       [ 0.98621653, -1.03959049],
       [-0.03630297,  1.00012316],
       [ 0.06452549,  2.92449032],
       [ 3.14254856, -0.93753166],
       [-0.05971118,  3.01663287],
       [-0.95237891,  2.96261688],
       [ 1.01428349, -0.93529517],
       [ 1.15784138,

In [35]:
melhoresFx

array([[1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319]])